In [84]:
import networkx as nx
from ortools.linear_solver import pywraplp
import sys
import random

# -----------------------------------------
# Section 1: Load and Prepare the Alternative Graph
# -----------------------------------------

def load_alternative_graph():
    G = nx.karate_club_graph()
    # Define possible occupations and interests
    occupations = ['Student', 'Engineer', 'Artist', 'Scientist', 'Teacher']
    interests = ['Music', 'Sports', 'Art', 'Technology', 'Travel', 'Reading', 'Gaming']

    # Assign attributes to nodes
    for node in G.nodes():
        G.nodes[node]['label'] = f"Person {node}"
        G.nodes[node]['age'] = random.randint(18, 65)
        G.nodes[node]['occupation'] = random.choice(occupations)
        G.nodes[node]['interests'] = random.sample(interests, k=2)  # Assign 2 random interests
    # Assign labels to edges
    for u, v in G.edges():
        G.edges[u, v]['predicate'] = 'friend'
        G.edges[u, v]['weight'] = random.uniform(0.5, 1.0)  # Assign a random weight between 0.5 and 1.0
        G.edges[u, v]['relationship_type'] = random.choice(['colleague', 'neighbor', 'classmate'])
    print(f"Graph loaded with {G.number_of_nodes()} nodes and {G.number_of_edges()} edges.")
    return G

# -----------------------------------------
# Section 2: Relevance-Based Filtering
# -----------------------------------------

def compute_simple_relevance(user_query, node_data):
    query_tokens = set(user_query.lower().split())
    relevance = 0

    # Consider label
    label_tokens = set(node_data.get('label', '').lower().split())
    relevance += len(query_tokens.intersection(label_tokens))

    # Consider occupation
    occupation = node_data.get('occupation', '').lower()
    if occupation in query_tokens:
        relevance += 1

    # Consider interests
    interests = [interest.lower() for interest in node_data.get('interests', [])]
    relevance += len(query_tokens.intersection(interests))

    return relevance

def filter_graph_by_relevance(G, user_query, top_k=1000):
    node_relevance = {}
    for node, data in G.nodes(data=True):
        relevance = compute_simple_relevance(user_query, data)
        if relevance > 0:
            node_relevance[node] = relevance

    if not node_relevance:
        print("No relevant nodes found. Please refine your query.")
        sys.exit()

    # Select top_k nodes
    sorted_nodes = sorted(node_relevance.items(), key=lambda x: x[1], reverse=True)[:top_k]
    selected_nodes = set(node for node, _ in sorted_nodes)

    # Include neighbors
    neighbors = set()
    for node in selected_nodes:
        neighbors.update(G.neighbors(node))
    selected_nodes.update(neighbors)

    # Create subgraph
    subgraph = G.subgraph(selected_nodes).copy()
    print(f"Subgraph extracted with {subgraph.number_of_nodes()} nodes and {subgraph.number_of_edges()} edges.")
    return subgraph, node_relevance

# -----------------------------------------
# Section 3: Compute Relevance Scores
# -----------------------------------------

def compute_relevance_scores(G, user_query):
    node_relevance = {}
    edge_relevance = {}
    query_tokens = set(user_query.lower().split())

    for node, data in G.nodes(data=True):
        relevance = compute_simple_relevance(user_query, data)
        node_relevance[node] = relevance / len(query_tokens) if query_tokens else 0

    for u, v, data in G.edges(data=True):
        u, v = sorted((u, v))
        node1_relevance = node_relevance.get(u, 0)
        node2_relevance = node_relevance.get(v, 0)
        # Edge predicate relevance
        predicate = data.get('predicate', '').lower()
        relationship_type = data.get('relationship_type', '').lower()
        predicate_tokens = set([predicate, relationship_type])
        common_tokens = query_tokens.intersection(predicate_tokens)
        predicate_relevance = len(common_tokens) / len(query_tokens) if query_tokens else 0
        relevance = (node1_relevance + node2_relevance + predicate_relevance) / 3
        edge_relevance[(u, v)] = relevance
    return node_relevance, edge_relevance

# -----------------------------------------
# Section 4: Define the MIP Model
# -----------------------------------------

def create_solver():
    solver = pywraplp.Solver.CreateSolver('CBC')
    if not solver:
        print('Solver not found.')
        sys.exit()
    return solver

def define_variables(solver, G):
    x = {}  # Node variables
    y = {}  # Edge variables

    for node in G.nodes:
        x[node] = solver.BoolVar(f'x[{node}]')

    for u, v in G.edges:
        u, v = sorted((u, v))
        y[(u, v)] = solver.BoolVar(f'y[{u},{v}]')

    return x, y

def set_objective_function(solver, x, y, node_relevance, edge_relevance):
    objective = solver.Objective()
    for node, var in x.items():
        relevance = node_relevance.get(node, 0)
        objective.SetCoefficient(var, relevance)
    for edge, var in y.items():
        relevance = edge_relevance.get(edge, 0)
        objective.SetCoefficient(var, relevance)
    objective.SetMaximization()

def add_constraints(solver, G, x, y, L_max, C_max):
    for u, v in G.edges:
        u, v = sorted((u, v))
        solver.Add(y[(u, v)] <= x[u])
        solver.Add(y[(u, v)] <= x[v])

    solver.Add(solver.Sum([var for var in y.values()]) <= L_max)

    solver.Add(
        solver.Sum([var for var in x.values()]) +
        solver.Sum([var for var in y.values()]) <= C_max
    )

    for node in G.nodes:
        incident_edges = []
        for edge in G.edges(node):
            u, v = sorted(edge)
            incident_edges.append(y[(u, v)])
        if incident_edges:
            solver.Add(x[node] <= solver.Sum(incident_edges))

# -----------------------------------------
# Section 5: Solve the Model
# -----------------------------------------

def solve_model(solver, time_limit=60000, mip_gap=0.01):
    solver.SetTimeLimit(time_limit)  # Time limit in milliseconds
    # Set the MIP gap parameter for the CBC solver
    solver.SetSolverSpecificParametersAsString(f'mipgap={mip_gap}')
    status = solver.Solve()
    return status

# -----------------------------------------
# Section 6: Extract and Use the Results
# -----------------------------------------

def extract_results(status, solver, x, y, G, node_relevance, edge_relevance):
    if status == pywraplp.Solver.OPTIMAL or status == pywraplp.Solver.FEASIBLE:
        if status == pywraplp.Solver.OPTIMAL:
            print('Optimal solution found.')
        else:
            print('Feasible solution found (may not be optimal).')

        selected_nodes = [node for node, var in x.items() if var.solution_value() > 0.5]
        selected_edges = [edge for edge, var in y.items() if var.solution_value() > 0.5]

        # Extract the largest connected component
        subgraph = G.subgraph(selected_nodes).copy()
        if len(subgraph.nodes) == 0:
            print('No nodes selected.')
            return None
        components = list(nx.connected_components(subgraph))
        largest_component = max(components, key=len)
        final_nodes = largest_component
        final_edges = [edge for edge in selected_edges if edge[0] in final_nodes and edge[1] in final_nodes]

        # Print selected nodes and edges
        print('Selected Nodes:')
        for node in final_nodes:
            data = G.nodes[node]
            label = data.get('label', '')
            occupation = data.get('occupation', '')
            interests = ', '.join(data.get('interests', []))
            relevance = node_relevance.get(node, 0)
            print(f'{node}: {label}, Occupation: {occupation}, Interests: {interests} (Relevance: {relevance:.4f})')

        print('\nSelected Edges:')
        for edge in final_edges:
            data = G.edges[edge]
            predicate = data.get('predicate', '')
            relationship_type = data.get('relationship_type', '')
            relevance = edge_relevance.get(edge, 0)
            print(f'{edge}: {predicate}, Type: {relationship_type} (Relevance: {relevance:.4f})')

        # Compute total relevance
        total_relevance = sum(node_relevance.get(node, 0) for node in selected_nodes) + \
                          sum(edge_relevance.get(edge, 0) for edge in selected_edges)
        print(f'\nTotal Relevance: {total_relevance:.4f}')

        # Prepare RAG input
        rag_input = ' '.join([G.nodes[node].get('label', '') for node in final_nodes])
        print(f'\nRAG Input: {rag_input}')

        return rag_input
    else:
        print('No optimal solution found.')
        print('Solver status code:', status)
        return None

# -----------------------------------------
# Section 7: Main Function
# -----------------------------------------

def main():
    # Define the user query
    user_query = "Find engineers interested in sports and technology who are friends of Person 0"

    # Load the knowledge graph
    G = load_alternative_graph()
    if G.number_of_nodes() == 0:
        print("The graph is empty. Exiting.")
        sys.exit()

    # Filter the graph
    subgraph, initial_node_relevance = filter_graph_by_relevance(G, user_query, top_k=1000)

    # Compute relevance scores on the subgraph
    node_relevance, edge_relevance = compute_relevance_scores(subgraph, user_query)

    # Create the solver
    solver = create_solver()

    # Define variables
    x, y = define_variables(solver, subgraph)

    # Set the objective function
    set_objective_function(solver, x, y, node_relevance, edge_relevance)

    # Add constraints
    L_max = 20  # Adjust as needed
    C_max = 40  # Adjust as needed
    add_constraints(solver, subgraph, x, y, L_max, C_max)

    # Solve the model
    status = solve_model(solver, time_limit=60000, mip_gap=0.01)

    # Extract results
    rag_input = extract_results(status, solver, x, y, subgraph, node_relevance, edge_relevance)

if __name__ == "__main__":
    main()


Graph loaded with 34 nodes and 78 edges.
Subgraph extracted with 34 nodes and 78 edges.
Optimal solution found.
Selected Nodes:
0: Person 0, Occupation: Scientist, Interests: Art, Music (Relevance: 0.1538)
1: Person 1, Occupation: Teacher, Interests: Art, Technology (Relevance: 0.1538)
2: Person 2, Occupation: Teacher, Interests: Reading, Sports (Relevance: 0.1538)
3: Person 3, Occupation: Teacher, Interests: Travel, Technology (Relevance: 0.1538)
5: Person 5, Occupation: Engineer, Interests: Technology, Reading (Relevance: 0.1538)
6: Person 6, Occupation: Student, Interests: Technology, Art (Relevance: 0.1538)
7: Person 7, Occupation: Student, Interests: Travel, Reading (Relevance: 0.0769)
9: Person 9, Occupation: Student, Interests: Technology, Travel (Relevance: 0.1538)
10: Person 10, Occupation: Engineer, Interests: Technology, Music (Relevance: 0.1538)
12: Person 12, Occupation: Teacher, Interests: Sports, Travel (Relevance: 0.1538)
15: Person 15, Occupation: Artist, Interests: Te